<a href="https://colab.research.google.com/github/ValdazoAmerico/distilbert-classificator/blob/main/distilbert_classificator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ktrain

     |████████████████████████████████| 25.3MB 121kB/s 
     |████████████████████████████████| 6.8MB 43.7MB/s 
     |████████████████████████████████| 983kB 33.0MB/s 
     |████████████████████████████████| 266kB 53.0MB/s 
     |████████████████████████████████| 1.9MB 48.8MB/s 
     |████████████████████████████████| 1.2MB 48.0MB/s 
     |████████████████████████████████| 471kB 52.2MB/s 
     |████████████████████████████████| 3.3MB 33.6MB/s 
     |████████████████████████████████| 901kB 45.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282390 sha256=1fc960130c5337367d9155a770b928c63e8598c9f4af4c2caccba184fe2a434b
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=f3d27d7c7aad2dcdbec5fbee2c920ad9448921e2070e684ba13f959a7d339ea9
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [ ]:
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups

In [ ]:
categories = ['alt.atheism','soc.religion.christian', 'comp.graphics', 'sci.med']

In [ ]:
train = fetch_20newsgroups(
    subset = 'train',
    categories=categories,
    shuffle = True,
    random_state = 0
)

In [ ]:
test = fetch_20newsgroups(
    subset = 'test',
    categories=categories,
    shuffle = True,
    random_state = 0
)

In [ ]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
test.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [ ]:
X_train = train.data
y_train = train.target

X_test = test.data
y_test = test.target

In [ ]:
Model_name = 'distilbert-base-uncased'

trans = text.Transformer(Model_name, maxlen=512, class_names=categories)

In [ ]:
train_data = trans.preprocess_train(X_train,y_train)
test_data = trans.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 343
	95percentile : 979
	99percentile : 2562


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=16)

In [ ]:
# learner.lr_find(show_plot=True, max_epochs=10)

simulating training for different learning rates... this may take a few moments...
Epoch 1/10
 71/141 [==============>...............] - ETA: 58:34 - loss: 1.3892 - accuracy: 0.2192

In [ ]:
learner.fit_onecycle(1e-4, 1)



begin training using onecycle policy with max lr of 0.0001...
142/142 [==============================] - 173s 1s/step - loss: 0.4904 - accuracy: 0.8285 - val_loss: 0.2746 - val_accuracy: 0.9101


In [ ]:
learner.validate(class_names=categories)

                        precision    recall  f1-score   support

           alt.atheism       0.89      0.74      0.81       319
soc.religion.christian       0.95      0.97      0.96       389
         comp.graphics       0.97      0.94      0.96       396
               sci.med       0.84      0.96      0.89       398

              accuracy                           0.91      1502
             macro avg       0.91      0.90      0.90      1502
          weighted avg       0.91      0.91      0.91      1502



array([[237,   6,   9,  67],
       [  8, 376,   1,   4],
       [  9,  12, 372,   3],
       [ 13,   2,   1, 382]])

In [ ]:
learner.view_top_losses(n=5, preproc=trans)

----------
id:1029 | loss:5.3 | true:comp.graphics | pred:soc.religion.christian)

----------
id:48 | loss:5.28 | true:alt.atheism | pred:soc.religion.christian)

----------
id:1132 | loss:5.26 | true:comp.graphics | pred:soc.religion.christian)

----------
id:1410 | loss:5.25 | true:alt.atheism | pred:comp.graphics)

----------
id:1490 | loss:5.22 | true:comp.graphics | pred:soc.religion.christian)



In [ ]:
X_test[311]

'From: klaus@ipri.go.jp (Klaus Hofmann;(6663))\nSubject: cats and pregnancy\nNntp-Posting-Host: 150.29.28.9\nOrganization: National Institute of Materials and Chem. Res., MITI, tsukuba\nLines: 13\n\nHello,\nI heard that a certain disease (toxoplasmosys?) is transmitted by cats which\ncan harm the unborn fetus. Does anybody know about it? Is it a problem to \nhave a cat in the same apartment?\n\nThanks\n\n\n\n-- \nKlaus Hofmann\nNational Institute of Materials and Chemical Research\n1-1, Higashi Tsukuba, Ibaraki 305, Japan\n'

# Predict on new data

In [ ]:
predictor = ktrain.get_predicteor(learner.model, preproc=trans)

In [ ]:
x = "I have a 42 yr old male friend, misdiagnosed as having osteopporosis for two years, who recently found out that his illnes is the rare Gaucher's disease."

In [ ]:
predictor.predict(x)

In [ ]:
predictor.explain(x)

/usr/local/lib/python3.7/dist-packages/ktrain/text/predictor.py:115: UserWarning: ktrain requires a forked version of eli5 to support tf.keras. Install with: pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1
  warnings.warn(msg)
